In [1]:
#本次优化目标：
#1.筛选出质量糟糕的题目，并且之后提交给后端进行隐藏
#2.建立一个关键词库，方便后端后续进行持续的识别
#3.设计一个新的rank优化标准

In [551]:
import pandas as pd
import pymysql
import re

import time
import datetime

In [736]:
pd.set_option('max_colwidth', 200)

In [586]:
df = pd.read_csv('C:/Users/ggg22/Desktop/iq.csv')
df_revise = df.copy(deep=True)

In [587]:
# 搜索函数
def search_col(df, col_name, string_list):
    if col_name == 'description':
        for i in string_list:
            print('successful find %s' % i)
            return(df[df.description.str.match(i)])#.sample(samples))
    elif col_name == 'selections':
        for i in string_list:
            print('successful find %s' % i)
            return(df[df.selections.str.match(i)])#.sample(samples))

# 删除函数
def delete_col(df, col_name, string_list = None, index = None): 
    if col_name == 'description':
        for i in string_list:
            df.drop(index = df[df.description.str.match(i)].index, inplace=True)
            print('successful delete %s' % i)
    elif col_name == 'selections':
        for i in string_list:
            df.drop(index = df[df.selections.str.match(i)].index, inplace=True)
            print('successful delete %s' % i)
    elif col_name == 'index':
        df.drop(index, inplace=True)

In [ ]:
# 首先进行20次随机取样，发现一些和垃圾题目相关的关键词
df_revise.sample(10)

In [555]:
# 验证题干以数字开头的题目质量
search_col(df_revise, 'description', [r'^\d.*']).sample(10)
# 经过10次随机取样后发现使用数字开头判断准确率并不高，但是发现可以通过其他方式判断

In [588]:
# 验证选项中仅包含“对”或“错”关键词的题目
search_col(df_revise, 'selections', [r'.*"对\\.*']).sample(10)

# 经过10次随机取样后发现基本都是垃圾题目，可以删除；后续还发现用户会使用✔、✘、x
search_col(df_revise, 'selections', [r'.*"对\\.*|.*"错\\.*', r'.*"✔\\.*|.*"x\\.*']).sample(frac = 1)

# 通过上述字符可以删除 10k+ 条垃圾题目，占总题目数的 14.72%
delete_col(df_revise ,'selections', [r'.*"对\\.*|.*"错\\.*', r'.*"✔\\.*|.*"x\\.*'])

successful find .*"对\\.*
successful find .*"对\\.*|.*"错\\.*
successful delete .*"对\\.*|.*"错\\.*
successful delete .*"✔\\.*|.*"x\\.*


In [589]:
# 验证题干中包含“答案”关键词
search_col(df_revise, 'description', [r'.*答案.*']).sample(10)

# 发现题干包含“答案”字符的非图片题基本都是垃圾
search_col(df_revise, 'description', [r'.*答案.*'])
tmp_cache = search_col(df_revise, 'description', [r'.*答案.*']).copy(deep=True)
tmp_cache.where(df_revise['type'] != 'image', inplace = True)
tmp_cache.dropna(how='all')

# 通过该字符排除图片题后，可以删除 180 条垃圾题目，占总题目数的 0.26%
delete_col(df_revise ,'index', index = tmp_cache.dropna(how='all').index)

successful find .*答案.*
successful find .*答案.*
successful find .*答案.*


In [609]:
# 验证选项中仅包含“不知道”关键词的题目
search_col(df_revise, 'selections', [r'.*不知道.*']).sample(10)

# 通过上述字符可以删除 2k+ 条垃圾题目，占总题目数的 3.31%
delete_col(df_revise ,'selections', [r'.*不知道.*'])

successful find .*不知道.*
successful delete .*不知道.*


In [630]:
# 验证选项中仅包含“单个数字”关键词的题目
search_col(df_revise, 'selections', [r'.*Text\\":\\"\d\\.*']).sample(10)

# 通过上述字符可以删除 2k+ 条垃圾题目，占总题目数的 3.35%
delete_col(df_revise ,'selections', [r'.*Text\\":\\"\d\\.*'])

successful find .*Text\\":\\"\d\\.*
successful delete .*Text\\":\\"\d\\.*


In [721]:
# 题干包含“选” - 195
search_col(df_revise, 'description', [r'.*选.*'])

# 题干包含字母 - 292
search_col(df_revise, 'description', [r'.*[a-dA-D].*']) 

# 排版错误 - 1057
search_col(df_revise, 'description', [r'.*\n.*'])

# 题干包含"脑筋急转弯" - 392
search_col(df_revise, 'description', [r'.*脑筋急转弯.*'])

# 题干包含“-” - 148
search_col(df_revise, 'description', [r'.*-.*'])

# 题干包含“超过4个数字” - 244
search_col(df_revise, 'description', [r'.*\d\d\d\d\d*.*'])

# 选项包含“选” - 15
search_col(df_revise, 'selections', [r'.*选\\.*'])

# 选项包含“ABCD” - 292
search_col(df_revise, 'selections', [r'.*Text\\":\\"[a-dA-D]\\.*'])

# 选项包含“。” - 292
search_col(df_revise, 'selections', [r'.*Text\\":\\"。\\.*'])

# 删除
search_col(df_revise, 'description', [r'.*选.*', r'.*[a-dA-D].*', r'.*\n.*', r'.*脑筋急转弯.*', r'.*-.*', r'.*\d\d\d\d\d*.*'])
delete_col(df_revise ,'selections', [r'.*选\\.*', r'.*Text\\":\\"[a-dA-D]\\.*', r'.*Text\\":\\"。\\.*'])

successful find .*选.*
successful find .*[a-dA-D].*
successful find .*\n.*
successful find .*脑筋急转弯.*
successful find .*-.*
successful find .*\d\d\d\d\d*.*
successful find .*选\\.*
successful find .*Text\\":\\"[a-dA-D]\\.*
successful find .*Text\\":\\"。\\.*
successful find .*选.*
successful delete .*选\\.*
successful delete .*Text\\":\\"[a-dA-D]\\.*
successful delete .*Text\\":\\"。\\.*


In [727]:
# 通过关键词共优化，粗略减少16499个垃圾题目，占比为23.57%
len(df.index) - len(df_revise.index)

16163

In [747]:
# 分数占比，举报为负分

#df_revise.where(df_revies[''])

# 举报 reports_count
tmp_cache = df_revise.copy(deep=True)
tmp_cache.where(df_revise['reports_count'] > 0, inplace = True)
tmp_cache.dropna(how='all').index

# 点赞 count_likes
tmp_cache = df_revise.copy(deep=True)
tmp_cache.where(df_revise['count_likes'] > 0, inplace = True)
tmp_cache.dropna(how='all').index

Int64Index([    3,     4,     6,    11,    12,    15,    16,    19,    20,
               21,
            ...
            68329, 68684, 68691, 68693, 68694, 68696, 68702, 68704, 68705,
            68709],
           dtype='int64', length=6853)